In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/AI RESIDENCY/P1')

Mounted at /content/drive


In [2]:
## Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns

## ignore warnings
import warnings
warnings.filterwarnings("ignore",module = "matplotlib\..*")

In [3]:
## Load data
data = pd.read_csv('data.csv')
data.head()

,Unnamed: 0,Happy,delivered_on_time,contents_as_expected,ordered_everything_I_wanted,paid_a_good_price,satisfied_with_my_courier,app_makes_ordering_easy
0,0,0,3,3,3,4,2,4
1,1,0,3,2,3,5,4,3
2,2,1,5,3,3,3,3,5
3,3,0,5,4,3,3,3,5
4,4,0,5,4,3,3,3,5


In [4]:
data = data.drop('Unnamed: 0',axis=1)
data.columns

Index(['Happy', 'delivered_on_time', 'contents_as_expected',
       'ordered_everything_I_wanted', 'paid_a_good_price',
       'satisfied_with_my_courier', 'app_makes_ordering_easy'],
      dtype='object')

In [5]:
## Let's try a neural network

import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print(device)

cpu


In [43]:
cols2drop = ['app_makes_ordering_easy','contents_as_expected','satisfied_with_my_courier','delivered_on_time']
df = data.copy()
for col in cols2drop:
  df = df.drop(col,axis=1)

In [44]:
#df = data.copy()
y = df['Happy'] ## response 
X = df.drop('Happy',axis=1) ## design matrix

from sklearn.model_selection import train_test_split

# split the data into training and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, ## data-features to split
                                                    y, ## data-response to split
                                                    test_size=0.1, ## we will leave 10% to test our models on later
                                                    random_state=42, ## make our work reproducible
                                                    shuffle=True) ## prevent data ordering affecting our models

In [45]:
## Defining the network

class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape, 30)
    self.fc2 = nn.Linear(30, 1)
    
    #self.dropout = nn.Dropout(0.25)

  def forward(self,x):
    x = torch.relu(self.fc1(x))
    #x = self.dropout(x)
    x = torch.sigmoid(self.fc2(x))
    
    return x

In [46]:
#hyper parameters
learning_rate = 0.01
epochs = 150
# Model , Optimizer, Loss
model = Net(input_shape=X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [47]:
#forward loop
losses = []
accur = []

x_train = torch.from_numpy(X_train.values).float()
y_train_ = torch.from_numpy(y_train.values).float()
for i in range(epochs):
     
    #calculate output
    output = model(x_train)
 
    #calculate loss
    loss = loss_fn(output, y_train_.reshape(-1,1))
 
    #accuracy
    predicted = model(torch.tensor(x_train))
    predicted[predicted >= 0.5] = 1
    predicted[predicted < 0.5] = 0 

    acc = (predicted == y_train_).sum() / len(y_train_)
    
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i%10 == 0:
      losses.append(loss)
      accur.append(acc)
      print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))
      #print("epoch {}\tloss : {}".format(i,loss))

epoch 0	loss : 0.7117190957069397	 accuracy : 51.610618591308594
epoch 10	loss : 0.683859646320343	 accuracy : 60.353981018066406
epoch 20	loss : 0.6776213049888611	 accuracy : 60.69911575317383
epoch 30	loss : 0.676063597202301	 accuracy : 62.1946907043457
epoch 40	loss : 0.6735743284225464	 accuracy : 61.1592903137207
epoch 50	loss : 0.6714015603065491	 accuracy : 61.1592903137207
epoch 60	loss : 0.6688798666000366	 accuracy : 62.1946907043457
epoch 70	loss : 0.6663288474082947	 accuracy : 62.1946907043457
epoch 80	loss : 0.6637898683547974	 accuracy : 62.079647064208984
epoch 90	loss : 0.6616679430007935	 accuracy : 62.079647064208984
epoch 100	loss : 0.6598585247993469	 accuracy : 62.079647064208984
epoch 110	loss : 0.6568368673324585	 accuracy : 59.43362808227539
epoch 120	loss : 0.6547908186912537	 accuracy : 59.31858444213867
epoch 130	loss : 0.6522904634475708	 accuracy : 59.43362808227539
epoch 140	loss : 0.6501268148422241	 accuracy : 59.43362808227539


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


In [48]:
## predictions
x_test = torch.from_numpy(X_test.values).float()
y_test_ = torch.from_numpy(y_test.values).float()

y_pred = model(x_test)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0


print(f'accuracy: {(y_pred.squeeze() == y_test_).sum() / len(y_test_)}')

accuracy: 0.692307710647583
